In [2]:
# чтение данных из файла
import pandas as pd
from sklearn.metrics import mean_squared_error
import joblib
import json
import yaml
import os
from sklearn.model_selection import train_test_split

df = pd.read_csv('dataframe/churn_data.csv')

df[:2]

,id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,online_security,...,device_protection,tech_support,streaming_tv,streaming_movies,gender,senior_citizen,partner,dependents,multiple_lines,target
0,7020,2017-10-01,NaN,Month-to-month,No,Bank transfer (automatic),64.45,1867.6,DSL,No,...,Yes,Yes,No,No,Female,1,No,No,Yes,0
1,7021,2018-10-01,NaN,Month-to-month,No,Electronic check,69.65,1043.3,Fiber optic,No,...,No,No,No,No,Male,1,No,No,No,0


In [3]:
# чтение данных из базы

import os, psycopg
from dotenv import load_dotenv
load_dotenv()

TABLE_NAME = "users_churn"# таблица с данными

connection = {"sslmode": "require", "target_session_attrs": "read-write"}
postgres_credentials = {
    "host": os.getenv("DB_DESTINATION_HOST"),
    "port": os.getenv("DB_DESTINATION_PORT"),
    "dbname": os.getenv("DB_DESTINATION_NAME"),
    "user": os.getenv("DB_DESTINATION_USER"),
    "password": os.getenv("DB_DESTINATION_PASSWORD"),
}

connection.update(postgres_credentials)

with psycopg.connect(**connection) as conn:

    with conn.cursor() as cur:
        cur.execute(f"SELECT * FROM {TABLE_NAME}")
        data = cur.fetchall()
        columns = [col[0] for col in cur.description]

df = pd.DataFrame(data, columns=columns)

df[:2]

,id,customer_id,begin_date,end_date,type,paperless_billing,payment_method,monthly_charges,total_charges,internet_service,...,device_protection,tech_support,streaming_tv,streaming_movies,gender,senior_citizen,partner,dependents,multiple_lines,target
0,2043,7361-YPXFS,2017-10-01,NaT,Month-to-month,No,Bank transfer (automatic),64.45,1867.6,DSL,...,Yes,Yes,No,No,Female,1,No,No,Yes,0
1,2044,6557-BZXLQ,2018-10-01,NaT,Month-to-month,No,Electronic check,69.65,1043.3,Fiber optic,...,No,No,No,No,Male,1,No,No,No,0


In [4]:
# трансформация данных (умная)
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import (
    OneHotEncoder, 
    SplineTransformer, 
    QuantileTransformer, 
    RobustScaler,
    PolynomialFeatures,
    KBinsDiscretizer,
)


# разделяем признаки на категории и числа
cat_columns = [
    'type',
    'paperless_billing',
    'payment_method',
    'internet_service',
    'online_security',
    'online_backup',
    'device_protection',
    'tech_support',
    'streaming_tv',
    'streaming_movies',
    'gender',
    'senior_citizen',
    'partner',
    'dependents',
    'multiple_lines'
]
num_columns = [
    'monthly_charges',
    'total_charges'
]

df[num_columns] = SimpleImputer(strategy='mean').fit_transform(df[num_columns])

n_knots = 3
degree_spline = 4
n_quantiles = 100
degree = 3
n_bins = 5
encode = 'ordinal'
strategy = 'uniform'
subsample = None

encoder_spl = SplineTransformer(n_knots=n_knots, degree=degree_spline)
encoder_q = QuantileTransformer(n_quantiles=n_quantiles)
encoder_rb = RobustScaler()
encoder_pol = PolynomialFeatures(degree=degree)
encoder_kbd = KBinsDiscretizer(n_bins=n_bins, encode=encode, strategy=strategy, subsample=subsample)

encoder_oh = OneHotEncoder(categories='auto', handle_unknown='ignore', max_categories=10, sparse_output=False, drop='first')

numeric_transformer = ColumnTransformer(
    transformers=[
        ('spl', encoder_spl, num_columns),
        ('q', encoder_q, num_columns),
        ('rb', encoder_rb, num_columns),
        ('pol', encoder_pol, num_columns),
        ('kbd', encoder_kbd, num_columns)
    ]
)

categorical_transformer = Pipeline(steps=[('encoder', encoder_oh)])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_columns),
        ('cat', categorical_transformer, cat_columns)
    ], n_jobs=-1
)

encoded_features = preprocessor.fit_transform(df)
transformed_df = pd.DataFrame(encoded_features, columns=preprocessor.get_feature_names_out())

transformed_df = pd.concat([transformed_df, df['target']], axis=1)
pd.set_option('display.max_columns', None)  # Отображать все колонки
transformed_df.head(10)

,num__spl__monthly_charges_sp_0,num__spl__monthly_charges_sp_1,num__spl__monthly_charges_sp_2,num__spl__monthly_charges_sp_3,num__spl__monthly_charges_sp_4,num__spl__monthly_charges_sp_5,num__spl__total_charges_sp_0,num__spl__total_charges_sp_1,num__spl__total_charges_sp_2,num__spl__total_charges_sp_3,num__spl__total_charges_sp_4,num__spl__total_charges_sp_5,num__q__monthly_charges,num__q__total_charges,num__rb__monthly_charges,num__rb__total_charges,num__pol__1,num__pol__monthly_charges,num__pol__total_charges,num__pol__monthly_charges^2,num__pol__monthly_charges total_charges,num__pol__total_charges^2,num__pol__monthly_charges^3,num__pol__monthly_charges^2 total_charges,num__pol__monthly_charges total_charges^2,num__pol__total_charges^3,num__kbd__monthly_charges,num__kbd__total_charges,cat__type_One year,cat__type_Two year,cat__paperless_billing_Yes,cat__payment_method_Credit card (automatic),cat__payment_method_Electronic check,cat__payment_method_Mailed check,cat__internet_service_Fiber optic,cat__internet_service_None,cat__online_security_Yes,cat__online_security_None,cat__online_backup_Yes,cat__online_backup_None,cat__device_protection_Yes,cat__device_protection_None,cat__tech_support_Yes,cat__tech_support_None,cat__streaming_tv_Yes,cat__streaming_tv_None,cat__streaming_movies_Yes,cat__streaming_movies_None,cat__gender_Male,cat__senior_citizen_1,cat__partner_Yes,cat__dependents_Yes,cat__multiple_lines_Yes,cat__multiple_lines_None,target
0,0.000002,0.056804,0.496757,0.416666,2.977230e-02,0.000000e+00,0.004502,0.232796,0.595606,0.165716,1.381000e-03,0.000000,0.440187,0.577099,-0.108556,0.138002,1.0,64.45,1867.60,4153.8025,120366.820,3.487930e+06,2.677126e+05,7.757642e+06,2.247971e+08,6.514058e+09,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0
1,0.000000,0.037981,0.446766,0.469639,4.561382e-02,1.142973e-08,0.014163,0.332225,0.556750,0.096732,1.302212e-04,0.000000,0.482560,0.420820,-0.012879,-0.105559,1.0,69.65,1043.30,4851.1225,72665.845,1.088475e+06,3.378807e+05,5.061176e+06,7.581228e+07,1.135606e+09,2.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0
2,0.037673,0.445748,0.470609,0.045970,1.595455e-08,0.000000e+00,0.037601,0.445511,0.470835,0.046053,1.718080e-08,0.000000,0.030303,0.128695,-0.935603,-0.375830,1.0,19.50,128.60,380.2500,2507.700,1.653796e+04,7.414875e+03,4.890015e+04,3.224902e+05,2.126782e+06,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0
3,0.000000,0.000030,0.076385,0.531653,3.715967e-01,2.033488e-02,0.000000,0.000270,0.111803,0.570269,3.066895e-01,0.010968,0.972832,0.971421,0.738730,1.789075,1.0,110.50,7455.45,12210.2500,823827.225,5.558373e+07,1.349233e+06,9.103291e+07,6.142003e+09,4.144018e+11,4.0,4.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0
4,0.024051,0.391047,0.517404,0.067486,1.131054e-05,0.000000e+00,0.041440,0.457652,0.459014,0.041894,1.432327e-13,0.000000,0.189033,0.023407,-0.839926,-0.406530,1.0,24.70,24.70,610.0900,610.090,6.100900e+02,1.506922e+04,1.506922e+04,1.506922e+04,1.506922e+04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0
5,0.000000,0.019105,0.364548,0.536516,7.979011e-02,4.100195e-05,0.034914,0.436287,0.479445,0.049353,1.456293e-07,0.000000,0.584610,0.169597,0.129715,-0.352916,1.0,77.40,206.15,5990.7600,15956.010,4.249782e+04,4.636848e+05,1.234995e+06,3.289331e+06,8.760926e+06,2.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
6,0.000000,0.001517,0.169995,0.596467,2.278294e-01,4.191700e-03,0.003433,0.214624,0.598159,0.181848,1.935148e-03,0.000000,0.838699,0.596864,0.486661,0.186076,1.0,96.80,2030.30,9370.2400,196533.040,4.122118e+06,9.070392e+05,1.902440e+07,3.990210e+08,8.369136e+09,3.0,1.0,0.0,0.0,1.

In [5]:
# разделяем выборки для обучения и для тестирования
X_train, X_test, y_train, y_test = train_test_split(
    transformed_df.drop(columns=['target']), transformed_df['target'], test_size=0.2, random_state=42
)
print(X_train[:2])
print(y_train[:2])

      num__spl__monthly_charges_sp_0  num__spl__monthly_charges_sp_1  \
2142                        0.000000                        0.000974   
1623                        0.022655                        0.384040   

      num__spl__monthly_charges_sp_2  num__spl__monthly_charges_sp_3  \
2142                        0.151139                        0.591574   
1623                        0.522678                        0.070610   

      num__spl__monthly_charges_sp_4  num__spl__monthly_charges_sp_5  \
2142                        0.250583                         0.00573   
1623                        0.000017                         0.00000   

      num__spl__total_charges_sp_0  num__spl__total_charges_sp_1  \
2142                      0.008657                      0.284808   
1623                      0.004649                      0.235077   

      num__spl__total_charges_sp_2  num__spl__total_charges_sp_3  \
2142                      0.580021                      0.126050   
1623    

In [6]:
# обучение модели

from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from category_encoders import CatBoostEncoder
from catboost import CatBoostClassifier
from sklearn.pipeline import Pipeline

model = CatBoostClassifier(auto_class_weights='Balanced', verbose=0)


model.fit(X_train, y_train) 

print (model)

In [7]:
prediction = model.predict(X_test)
prediction_proba = model.predict_proba(X_test)[:, 1]
print(prediction)
print(prediction_proba)

[1 0 0 ... 0 0 1]
[0.59138702 0.3490331  0.02720655 ... 0.03580795 0.01079301 0.60560879]


In [8]:
# Вычисление метрик
from sklearn.metrics import f1_score, roc_auc_score

f1 = f1_score(y_test, prediction)
roc_auc = roc_auc_score(y_test, prediction_proba)

print(f"F1 Score: {f1}")
print(f"ROC AUC: {roc_auc}")

F1 Score: 0.6214039125431531
ROC AUC: 0.8446179467902937


In [9]:
# Сохранение модели и метрик

import os
import mlflow
import mlflow.sklearn
import pandas as pd

EXPERIMENT_NAME = "churn"
RUN_NAME = "model_transformed_data"
REGISTRY_MODEL_NAME = "churn_model_arvas"


os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net"
os.environ["AWS_ACCESS_KEY_ID"] = "YCAJE3Nlz8iDILW5VTYM1ihQB"
os.environ["AWS_SECRET_ACCESS_KEY"] = "YCPjvS7uwhvJpUj3bKm8X-IX4QAwBIVsvX61IL44"

mlflow.set_tracking_uri("http://localhost:5000")

pip_requirements = 'requirements.txt'
signature = mlflow.models.infer_signature(X_test, prediction)
input_example = X_test[:10]
metadata = {'model_learn': 'transformed_data'}

print (model)
experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id
    # ваш код здесь
    model_info = mlflow.catboost.log_model(
        cb_model=model,  # Ваш обученный экземпляр модели DecisionTreeClassifier
        artifact_path='models',
        registered_model_name=REGISTRY_MODEL_NAME,
        signature=signature,
        input_example=input_example,
        metadata=metadata,
        pip_requirements=pip_requirements,
        await_registration_for=60
    )
    # Логирование метрик
    mlflow.log_metric("F1_Score", f1)
    mlflow.log_metric("ROC_AUC", roc_auc)


Registered model 'churn_model_arvas' already exists. Creating a new version of this model...
2025/09/13 12:47:41 INFO mlflow.tracking._model_registry.client: Waiting up to 60 seconds for model version to finish creation. Model name: churn_model_arvas, version 5
Created version '5' of model 'churn_model_arvas'.


In [19]:
# Сохранение модели

import os
import mlflow
import mlflow.sklearn
import pandas as pd

EXPERIMENT_NAME = "churn"
RUN_NAME = "model_0_registry"
REGISTRY_MODEL_NAME = "churn_model_arvas"


os.environ["MLFLOW_S3_ENDPOINT_URL"] = "https://storage.yandexcloud.net"
os.environ["AWS_ACCESS_KEY_ID"] = "YCAJE3Nlz8iDILW5VTYM1ihQB"
os.environ["AWS_SECRET_ACCESS_KEY"] = "YCPjvS7uwhvJpUj3bKm8X-IX4QAwBIVsvX61IL44"

mlflow.set_tracking_uri("http://localhost:5000")

X_transformed_test = pipeline.named_steps['preprocessor'].transform(X_test)

pip_requirements = 'requirements.txt'
signature = mlflow.models.infer_signature(X_transformed_test, prediction)
input_example = X_transformed_test[:10]
metadata = {'model_type': 'monthly'}

print (model)
experiment_id = mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    run_id = run.info.run_id
    # ваш код здесь
    model_info = mlflow.catboost.log_model(
        cb_model=model,  # Ваш обученный экземпляр модели DecisionTreeClassifier
        artifact_path='models',
        registered_model_name=REGISTRY_MODEL_NAME,
        signature=signature,
        input_example=input_example,
        metadata=metadata,
        pip_requirements=pip_requirements,
        await_registration_for=60
    )

Registered model 'churn_model_arvas' already exists. Creating a new version of this model...
2025/09/10 05:38:56 INFO mlflow.tracking._model_registry.client: Waiting up to 60 seconds for model version to finish creation. Model name: churn_model_arvas, version 2
Created version '2' of model 'churn_model_arvas'.
